In [1]:
import numpy as np
import pandas as pd

# Coefficients from the model 5
coef = {
    'AMARELA': 0.702,
    'BRANCA': 0.421,  # White advantage over Black (reference)
    'FEMININO': -0.897,
    'FUNDO_ESPECIAL': 0.034,
    'PARDA': 0.310,
    'INDIGENA': 0.0817,  # Keeping from previous model
    'Constante': -3.7351,  # Keeping constant from previous model
    # Other controls
    'ENSINO_FUNDAMENTAL_COMPLETO': -0.605,
    'GOVERNADOR': 0.957,
    'LOG_EDUCACAO': -0.157,
    'LOG_SAUDE_ASSIST_SOCIAL': 0.142,
    'PRESIDENTE': 0.968,
    'REELEICAO': 2.356
}

def calculate_equivalent_funding(target_group, reference_group, coefficients):
    """Calculate how much more funding a group needs to match the reference group's election probability."""
    
    def group_effect(race, gender):
        coef_race = coefficients.get(race, 0)  # 0 for PRETA (Black)
        coef_gender = coefficients['FEMININO'] if gender == 'FEMININO' else 0
        return coef_race + coef_gender
    
    target_effect = group_effect(*target_group)
    reference_effect = group_effect(*reference_group)
    effect_difference = reference_effect - target_effect
    
    # Campaign fund coefficient
    beta_fund = coefficients['FUNDO_ESPECIAL']
    
    if beta_fund == 0:
        return (np.inf, np.inf)
    
    log_difference = effect_difference / beta_fund
    multiplier = np.exp(log_difference)
    
    return (log_difference, multiplier)

# Reference group (White male)
REFERENCE = ('BRANCA', 'MASCULINO')

# Groups for comparison
groups = [
    ('PRETA', 'MASCULINO'),     # Black male
    ('PRETA', 'FEMININO'),      # Black female
    ('BRANCA', 'FEMININO'),     # White female
    ('PARDA', 'MASCULINO'),     # Brown male
    ('PARDA', 'FEMININO'),      # Brown female
    ('AMARELA', 'MASCULINO'),   # Asian male
    ('AMARELA', 'FEMININO'),    # Asian female
    ('INDIGENA', 'MASCULINO'),  # Indigenous male
    ('INDIGENA', 'FEMININO')    # Indigenous female
]

# Calculate results
results = []
for group in groups:
    log_diff, multiplier = calculate_equivalent_funding(group, REFERENCE, coef)
    results.append({
        'Race': group[0],
        'Gender': group[1],
        'Δ log(Funding)': log_diff,
        'Multiplier': multiplier,
        'Interpretation': f"Equal" if multiplier == 1 else f"{multiplier:.1f}x funding"
    })

# Create comparison table
results_table = pd.DataFrame(results)

# Save to Excel file
output_path = r'funding_comparison.xlsx'
results_table.to_excel(output_path, index=False, float_format="%.2f")

print(f"Results saved successfully to: {output_path}")

# Print summary to console
print("\nFunding Needed to Match Election Probability of White Men:")
print(results_table.to_string(float_format="%.1f", index=False))

# Special cases
black_women_log_diff, black_women_mult = calculate_equivalent_funding(
    ('PRETA', 'FEMININO'), REFERENCE, coef)
white_women_log_diff, white_women_mult = calculate_equivalent_funding(
    ('BRANCA', 'FEMININO'), REFERENCE, coef)

print(f"\nKey Findings:")
print(f"- Black women need {black_women_mult:,.0f}x the funding of white men")
print(f"- White women need {white_women_mult:,.0f}x the funding of white men")

Results saved successfully to: funding_comparison.xlsx

Funding Needed to Match Election Probability of White Men:
    Race    Gender  Δ log(Funding)          Multiplier               Interpretation
   PRETA MASCULINO            12.4            238554.5            238554.5x funding
   PRETA  FEMININO            38.8 68438086336100648.0 68438086336100648.0x funding
  BRANCA  FEMININO            26.4      286886625390.3      286886625390.3x funding
   PARDA MASCULINO             3.3                26.2                26.2x funding
   PARDA  FEMININO            29.6     7508514982946.4     7508514982946.4x funding
 AMARELA MASCULINO            -8.3                 0.0                 0.0x funding
 AMARELA  FEMININO            18.1          73857421.7          73857421.7x funding
INDIGENA MASCULINO            10.0             21577.6             21577.6x funding
INDIGENA  FEMININO            36.4  6190329467030989.0  6190329467030989.0x funding

Key Findings:
- Black women need 68,438,086,